In [8]:
import sqlite3
import pandas as pd

query = """
WITH SocialContext AS (
    SELECT 
        frame_number,
        video_id,
        CASE
            WHEN MAX(CASE WHEN object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            AND MAX(CASE WHEN object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'with both children and adults'
            WHEN MAX(CASE WHEN object_class IN (1,3) THEN 1 ELSE 0 END) = 1 
            THEN 'with an adult'
            WHEN MAX(CASE WHEN object_class IN (0,2) THEN 1 ELSE 0 END) = 1 
            THEN 'with another child'
            ELSE 'alone'
        END as social_context
    FROM Detections
    GROUP BY frame_number, video_id
),
ObjectPresence AS (
    SELECT DISTINCT
        frame_number,
        video_id,
        1 as has_object,
        CASE 
            WHEN object_class = 5 THEN 'book'
            WHEN object_class = 6 THEN 'toy'
            WHEN object_class = 7 THEN 'kitchenware'
            WHEN object_class = 8 THEN 'screen'
            WHEN object_class = 9 THEN 'food'
            WHEN object_class = 10 THEN 'other_object'
        END as object_type
    FROM Detections
    WHERE object_class IN (5,6,7,8,9,10)
)

SELECT
    f.frame_id,
    f.video_id,
    COALESCE(op.object_type, 'none') as object_type,
    v.age_at_recording,
    COALESCE(sc.social_context, 'alone') as social_context,
    COALESCE(op.has_object, 0) as has_object
FROM 
    Frames f
    LEFT JOIN Videos v ON f.video_id = v.video_id
    LEFT JOIN SocialContext sc ON f.frame_number = sc.frame_number AND f.video_id = sc.video_id
    LEFT JOIN ObjectPresence op ON f.frame_number = op.frame_number AND f.video_id = op.video_id
ORDER BY 
    f.video_id, f.frame_id, op.object_type;
"""

with sqlite3.connect('/home/nele_pauline_suffo/outputs/detection_pipeline_results/detection_results.db') as conn:
    df = pd.read_sql_query(query, conn)

print(f"Retrieved {len(df)} rows")
display(df.head())

Retrieved 134210 rows


,frame_id,video_id,object_type,age_at_recording,social_context,has_object
0,1,1,none,3.96,with an adult,0
1,2,1,none,3.96,with an adult,0
2,3,1,none,3.96,with an adult,0
3,4,1,none,3.96,with an adult,0
4,5,1,none,3.96,with an adult,0


In [9]:
# Convert columns to categorical type
df['object_type'] = df['object_type'].astype('category')
df['social_context'] = df['social_context'].astype('category')

In [10]:
# export data to csv
df.to_csv('/home/nele_pauline_suffo/outputs/detection_pipeline_results/objects_detection_results.csv', index=False)